In [1]:
import pandas as pd

In [7]:
from openie import StanfordOpenIE

In [ ]:
import os
os.environ["CORENLP_HOME"] = r'D:\\Downloads\\stanford-corenlp-full-2018-10-05\\stanford-corenlp-full-2018-10-05'

In [82]:
import string
table = str.maketrans('', '', string.punctuation)

In [2]:
#imports the data
roughDat= pd.read_csv("../data/raw/Emergent_NAACL2016/emergent/url-versions-2015-06-14-clean-test-fold-1.csv")

In [3]:
#removes unnecessary columns.
roughDat = roughDat.drop(columns=['Unnamed: 0', 'claimId',
       'articleId', 'articleHeadlineStance'])

In [68]:
claims = roughDat["claimHeadline"].tolist()
headlines = roughDat["articleHeadline"].tolist()
n1=[]
n2=[]

In [9]:
with StanfordOpenIE() as client:
    for i in claims:
        n1.append(client.annotate(i))

Starting server with command: java -Xmx8G -cp /Users/nikilsaldanaha/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-479ecd83ce1b4a7e.props -preload openie


In [8]:
with StanfordOpenIE() as client:
    for i in headlines:
        n2.append(client.annotate(i))


Extracting to /Users/nikilsaldanaha/stanfordnlp_resources.
Starting server with command: java -Xmx8G -cp /Users/nikilsaldanaha/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-02fb25b418a64dfc.props -preload openie


In [10]:
len(n1)

177

In [11]:
len(n2)

177

In [16]:
n1[0]

[{'subject': 'Developers',
  'relation': 'have limited',
  'object': 'access to Apple Watch sensors'},
 {'subject': 'Developers', 'relation': 'have limited', 'object': 'access'}]

In [25]:
claim_subjects = []
claim_objects = []
claim_relations = []
for claim_svo in n1:
    t1 = set()
    t2 = set()
    t3 = set()
    for a in claim_svo:
        t1.add(a["subject"])
        t2.add(a["object"])
        t3.add(a["relation"])
    claim_subjects.append(t1)
    claim_objects.append(t2)
    claim_relations.append(t3)

In [27]:
headline_subjects = []
headline_objects = []
headline_relations = []
for headline_svo in n2:
    t1 = set()
    t2 = set()
    t3 = set()
    for a in headline_svo:
        t1.add(a["subject"])
        t2.add(a["object"])
        t3.add(a["relation"])
    headline_subjects.append(t1)
    headline_objects.append(t2)
    headline_relations.append(t3)

In [ ]:
Equivalence=[]
for m,n in zip(claimSVO,headSVO):
    if len(m)!=0 and len(n)!=0:
        l2=word_tokenize(n[0]["subject"].lower())
        l1=word_tokenize(m[0]["subject"].lower())
        if len(l1)!=0 and len(l2)!=0:
            for i in l1:
                if i in l2:
                    Equivalence.append("Equivalence")
                    break
                else:
                    Equivalence.append("No Relation")

# import these modules
from nltk.stem import PorterStemmer

ps = PorterStemmer()

In [35]:
from nltk.tokenize import word_tokenize

In [34]:
claim_objects[0], headline_subjects[0]

({'access', 'access to Apple Watch sensors'}, {'Apple Watch Developers'})

In [43]:
subject_equivalence = []
for cs, hs in zip(claim_subjects, headline_subjects):
    claim_tokens = set()
    headline_tokens = set()
    for sub in cs:
        claim_tokens.update(word_tokenize(sub.lower()))
    for sub in hs:
        headline_tokens.update(word_tokenize(sub.lower()))
    subject_equivalence.append(len(claim_tokens.intersection(headline_tokens)) == 0)

In [76]:
claim_subjects[12]

{'Seven underage Bosnian girls', 'underage Bosnian girls'}

In [78]:
import pickle

In [79]:
# load the ppdb data
with open("../data/processed/ppdb/ppdb-2.0-tldr-dict.pkl", "rb") as f:
    ppdb_dict = pickle.load(f)

In [94]:
ppdb_dict["apples"]

{'pears': (3.84409, 'OtherRelated')}

In [124]:
subject_entailments = []
for cs, hs in zip(claim_subjects, headline_subjects):
    l = []
    entailment = ""
    for subject in cs:
        sub = subject.lower().translate(table)
        l.append(ppdb_dict.get(sub, {}))
    done = False
    for subject in hs:
        for para in l:
            entailment=para.get(subject,"noRelation")
            if entailment!="noRelation":
                subject_entailments.append(entailment[1])
                done = True
                break
        if done is True:
            break
    subject_entailments.append("noRelation")

In [125]:
relation_entailments = []
for cr, hr in zip(claim_relations, headline_relations):
    l = []
    entailment = ""
    for relation in cr:
        rel = relation.lower().translate(table)
        l.append(ppdb_dict.get(rel, {}))
    done = False
    for relation in hr:
        for para in l:
            entailment = para.get(relation,"noRelation")
            if entailment != "noRelation":
                relation_entailments.append(entailment[1])
                done = True
                break
        if done is True:
            break
    relation_entailments.append("noRelation")

In [126]:
object_entailments = []
for cr, hr in zip(claim_objects, headline_objects):
    l = []
    entailment = ""
    for obj in cr:
        rel = obj.lower().translate(table)
        l.append(ppdb_dict.get(rel, {}))
    done = False
    for obj in hr:
        for para in l:
            entailment = para.get(obj,"noRelation")
            if entailment != "noRelation":
                object_entailments.append(entailment[1])
                done = True
                break
        if done is True:
            break
    object_entailments.append("noRelation")

In [127]:
for i, (x,y) in enumerate(zip(subject_entailments, subject_equivalence)):
    print(x, y)
    if (x == "noRelation" or x == "Independent") and y is True:
        subject_entailments[i] = "Equivalence"

noRelation False
noRelation True
noRelation False
noRelation False
noRelation False
noRelation True
noRelation True
noRelation True
noRelation False
noRelation True
noRelation True
noRelation True
noRelation False
noRelation True
noRelation True
noRelation True
noRelation False
noRelation False
noRelation False
noRelation False
noRelation True
noRelation True
noRelation True
noRelation True
noRelation False
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation False
noRelation False
noRelation False
noRelation False
noRelation True
noRelation True
noRelation True
noRelation True
noRelation False
noRelation True
noRelation True
noRelation False
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation True
noRelation False
noRela

In [128]:
subject_entailments

['noRelation',
 'Equivalence',
 'noRelation',
 'noRelation',
 'noRelation',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'noRelation',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'noRelation',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'noRelation',
 'noRelation',
 'noRelation',
 'noRelation',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'noRelation',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'noRelation',
 'noRelation',
 'noRelation',
 'noRelation',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'noRelation',
 'Equivalence',
 'Equivalence',
 'noRelation',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'Equivalence',
 'noRelation',
 'noRelation',
 'Equivalence',
 'noRelatio

In [129]:
equiv_map = {
    "Independent": 0,
    "OtherRelated": 1,
    "Equivalence": 2,
    "ForwardEntailment": 3,
    "ReverseEntailment": 4,
    "Exclusion": 5,
    "noRelation": 0
}

In [130]:
object_entailments = [equiv_map[i] for i in object_entailments]
relation_entailments = [equiv_map[i] for i in relation_entailments]
subject_entailments = [equiv_map[i] for i in subject_entailments]

In [135]:
import numpy as np

In [137]:
svo_feature = np.array(list(zip(subject_entailments, object_entailments, relation_entailments)))

In [138]:
np.savetxt("../data/processed/features/svo_feature.csv", svo_feature, delimiter=",")